In [127]:
using LinearAlgebra, SparseArrays
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()


In [128]:
gmsh.open("cantilever1D-a.geo")

Info    : Reading 'cantilever1D-a.geo'...
Info    : Done reading 'cantilever1D-a.geo'


In [ ]:
A = 1
l = 100
n = 2
F = 10
fx = 1

order = 1

3

In [130]:
gmsh.parser.setNumber("l", [l])
gmsh.parser.setNumber("A", [A])
gmsh.parser.setNumber("n", [n])
gmsh.parser.setNumber("order", [order])

In [131]:
gmsh.merge("cantilever1D-b.geo")

Info    : Reading 'cantilever1D-b.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000294892s, CPU 0.000284s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 7.4808e-05s, CPU 7.1e-05s)
Info    : Meshing order 3 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 3
Info    : [ 30%] Meshing curve 2 order 3
Info    : [ 50%] Meshing curve 3 order 3
Info    : [ 70%] Meshing curve 4 order 3
Info    : [ 90%] Meshing surface 1 order 3
Info    : Done meshing order 3 (Wall 0.000221634s, CPU 0.000221s)
Info    : 16 nodes 9 elements
Info    : Done reading 'cantilever1D-b.geo'


In [132]:
mat = FEM.material("body", E=2e5, ν=0)
problem = FEM.Problem([mat], type=:PlaneStress, bandwidth=:RCMK)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 15)


LowLevelFEM.Problem("cantilever1D-a", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.0, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 16)

In [133]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 1)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [134]:
K = FEM.stiffnessMatrix(problem)

32×32 SparseMatrixCSC{Float64, Int64} with 1021 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣽⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [135]:
supp1 = FEM.displacementConstraint("bottom", uy=0)
supp2 = FEM.displacementConstraint("left", ux=0)
load1 = FEM.load("right", fx=10)
#load2 = FEM.traction("bottom", fx=1)
#load3 = FEM.traction("top", fx=1)
load4 = FEM.load("body", fx=1);

In [136]:
f = FEM.loadVector(problem, [load1, load4])

32-element Vector{Float64}:
 14.062499999999993
  0.0
 14.062499999999973
  0.0
 14.06250000000001
  0.0
 14.06250000000001
  0.0
  4.687499999999988
  0.0
  ⋮
  0.0
  1.5625000000000007
  0.0
  2.8124999999999982
  0.0
  2.812499999999994
  0.0
  1.5624999999999987
  0.0

In [137]:
K1, f1 = FEM.applyBoundaryConditions(problem, K, f, [supp1, supp2])
K1

32×32 SparseMatrixCSC{Float64, Int64} with 583 stored entries:
⎡⣿⣿⣿⣿⢸⢸⣿⣿⣟⣿⡇⡇⢸⣿⡇⠀⎤
⎢⣿⣿⣿⣿⢸⢸⣿⣿⣿⣿⡇⡇⢸⣿⡇⠀⎥
⎢⣒⣒⣒⣒⢑⢔⣒⣒⣒⣒⡂⡂⢐⣒⡂⠀⎥
⎢⣿⣿⣿⣿⢸⢸⣿⣿⣿⣿⡇⡇⢸⣿⡇⠀⎥
⎢⣿⣿⣿⣿⢸⢸⣿⣿⣿⣿⡇⡇⢸⣿⡇⠀⎥
⎢⠭⠭⠭⠭⠨⠨⠭⠭⠭⠭⠕⢅⠨⠭⠅⠀⎥
⎢⣶⣶⣶⣶⢰⢰⣶⣶⣶⣶⡆⡆⢱⣶⡆⠀⎥
⎣⠉⠉⠉⠉⠈⠈⠉⠉⠉⠉⠁⠁⠈⠉⠑⢄⎦

In [138]:
q = FEM.solveDisplacement(K1, f1)

32-element Vector{Float64}:
  0.015555555555487058
  7.993816235248337e-20
  0.02555555555545935
 -2.4429070920757688e-19
  0.02555555555545935
 -1.5899194955106546e-19
  0.015555555555487056
  4.9352939195163495e-20
  0.0
 -1.2044712703527034e-19
  ⋮
  0.0
  0.0
 -2.93150708529777e-19
  0.029999999999897085
  5.661927832330837e-19
  0.029999999999897144
  0.0
  0.0
  0.0

In [139]:
FT = K * q - f

32-element Vector{Float64}:
  -9.458034355702694e-11
  -6.821210263296962e-13
   3.276845461641642e-11
  -2.2737367544323206e-13
  -1.0187406473960436e-10
  -6.821210263296962e-13
   3.6273206660553114e-12
   2.2737367544323206e-13
 -41.24999999978171
   8.526512829121202e-14
   ⋮
   1.1368683772161603e-13
 -13.749999999915644
  -1.0658141036401503e-14
   2.730260462158185e-12
   1.4210854715202004e-13
  -2.9097613207795803e-11
  -5.684341886080801e-13
 -13.749999999954523
  -6.252776074688882e-13

In [140]:
S = FEM.solveStress(problem, q)

LowLevelFEM.TensorField([[109.99999999942736; -4.3946328057815173e-13; … ; 0.0; 0.0;;]], [5], 1, :s)

In [141]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [142]:
ux = FEM.showDoFResults(problem, q, :ux, name="uₓ", visible=false)
sx = FEM.showStressResults(problem, S, :sx, name="N", visible=false, smooth=false)
#gmsh.view.option.setNumber(sx, "MaxRecursionLevel", 0)
ff = FEM.showDoFResults(problem, f, :uvec, name="f", visible=false)
ft = FEM.showDoFResults(problem, FT, :uvec, name="F_T", visible=true)

3

In [143]:
display("FT = $(sum(FT)) N")
display("N(0) = $(gmsh.view.probe(sx, 0.001, A/2, 0)[1][1]) N")

"FT = -109.99999999983125 N"

"N(0) = 109.99899999942713 N"

In [144]:
gmsh.view.option.setNumber(ft, "VectorType", 4)
gmsh.view.option.setNumber(ft, "GlyphLocation", 2)
gmsh.view.option.setNumber(ft, "ArrowSizeMax", 100)

gmsh.view.option.setNumber(ff, "VectorType", 4)
gmsh.view.option.setNumber(ff, "GlyphLocation", 2)
gmsh.view.option.setNumber(ff, "ArrowSizeMax", 100)

In [145]:
FEM.plotOnPath(problem, "path", sx, name="N", visible=true);
FEM.plotOnPath(problem, "path", ux, name="uₓ", visible=true);

In [146]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [147]:
gmsh.finalize()